In [2]:
import pandas as pd
import duckdb as db

In [3]:
df = pd.read_excel("C:/Users/elifs/OneDrive/Masaüstü/watson_healthcare_modified.xlsx")

In [4]:
print(df.columns)

Index(['EmployeeID', 'Age', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'Shift',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


In [5]:
db.query("""
    SELECT Round(AVG(age),2) AS avg_age
    FROM df
""").to_df()

,avg_age
0,36.87


In [6]:
db.query("""
    SELECT  EducationField, Round(AVG(age),2) AS avg_age
    FROM df
    GROUP BY EducationField
""").to_df()

,EducationField,avg_age
0,Marketing,37.82
1,Medical,36.93
2,Human Resources,37.21
3,Other,35.59
4,Life Sciences,36.88
5,Technical Degree,36.05


In [7]:
db.query("""
    SELECT JobRole, Round(AVG(age),2) AS avg_age
    FROM df
    GROUP BY JobRole
""").to_df()

,JobRole,avg_age
0,Other,35.33
1,Nurse,36.10
2,Administrative,45.41
3,Therapist,38.58
4,Admin,46.13


In [1]:
db.query("""
SELECT COUNT(DISTINCT EmployeeID) AS n_age
FROM df
WHERE age > (SELECT AVG(age) FROM df);
""").to_df()

NameError: name 'db' is not defined